### Objective:

Develop an algorithm which will identify the artist when provided with a drawing, with state of the art precision.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Dataset: https://www.kaggle.com/ikarus777/best-artworks-of-all-time

In [2]:
import numpy as np
import pandas as pd 
import cv2
from collections import Counter
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil
from glob import glob
%matplotlib inline

from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam 
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau, EarlyStopping

from keras.preprocessing.image import ImageDataGenerator


In [3]:
def plotImages(artist,directory):
    print(artist)
    multipleImages = glob(directory)
    plt.rcParams['figure.figsize'] = (15, 15)
    plt.subplots_adjust(wspace=0, hspace=0)
    i_ = 0
    for l in multipleImages[:25]:
        im = cv2.imread(l)
        im = cv2.resize(im, (128, 128)) 
        plt.subplot(5, 5, i_+1) #.set_title(l)
        plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB)); plt.axis('off')
        i_ += 1

### Display some paintaings by **ACo**

In [4]:
plotImages("ACo","../input/inducks-drawings-by-artist-us/CB/**")      

### Display some paintaings by **AFe**

In [5]:
plotImages("AFe","../input/inducks-drawings-by-artist-us/DR/**")      

### Display some paintaings by **AH**

In [6]:
plotImages("AH","../input/inducks-drawings-by-artist-us/AH/**")      

### Load the CSV data

In [7]:
artists = pd.read_csv('../input/inducks-drawings-by-artist-us-metadata/artists.csv')

In [8]:
artists

In [9]:
artists.shape

Let us do some data exploration
### Plot drawings by nationality of the cartoonist. 

In [10]:
plt.figure(figsize=(5,5))
nationalityPlot = sns.countplot(y='nationality',data=artists)
nationalityPlot

### Plot artists and count of drawings

In [11]:
figsize=(15, 5)
ticksize = 14
titlesize = ticksize + 8
labelsize = ticksize + 5

params = {'figure.figsize' : figsize,
          'axes.labelsize' : labelsize,
          'axes.titlesize' : titlesize,
          'xtick.labelsize': ticksize,
          'ytick.labelsize': ticksize}

plt.rcParams.update(params)

col1 = "name"
col2 = "drawings"

sns.barplot(x=col1, y=col2, data=artists)
plt.title("Drawing Count by Artist")
plt.xlabel("Artist")
plt.ylabel("Drawing Count")
plt.xticks(rotation=90)
plt.plot()

### Create dataframe with artists having min of 200 drawings.

Here I have first sorted data according to feature drawing so that all my data in sorted manner and according to their drawing name. I have taken drawings >200 for our analysis just taken sample data.


I have also added one more feature in this called ‘class_weight’ which put weight on drawings why we want weight actually simple putting weight on something it tells us the importance of that particular thing right.

In [12]:
# Sort artists by number of drawings
artists = artists.sort_values(by=['drawings'], ascending=False)

# Create a dataframe with artists having more than 200 drawings
artists_top = artists[artists['drawings'] >= 200].reset_index()
artists_top = artists_top[['personcode', 'drawings']]

artists_top['class_weight'] = artists_top.drawings.sum() / (artists_top.shape[0] * artists_top.drawings)
artists_top

Create a key value pairs of class index and weights

In [13]:
# Set class weights - assign higher weights to underrepresented classes
class_weights = artists_top['class_weight'].to_dict()
class_weights

In [14]:
# There is some problem recognizing 'Albrecht_Dürer' (don't know why, worth exploring)
# So I'll update this string as directory name to df's
#updated_name = "Albrecht_Dürer".replace("_", " ")
#artists_top.iloc[4, 0] = updated_name

In [15]:
# Explore images of top artists
images_dir = '../input/inducks-drawings-by-artist-us/'
artists_dirs = os.listdir(images_dir)
artists_top_name = artists_top['personcode'].values

# See if all directories exist
for name in artists_top_name:
    if os.path.exists(os.path.join(images_dir, name)):
        print("Found -->", os.path.join(images_dir, name))
    else:
        print("Did not find -->", os.path.join(images_dir, name))

### Print few random drawings along with artists name

Lets display some random drawings for clear visualization whats happening right.

In [16]:
# Print few random drawings
import random
n = 5
fig, axes = plt.subplots(1, n, figsize=(20,10))

for i in range(n):
    random_artist = random.choice(artists_top_name)
    random_image = random.choice(os.listdir(os.path.join(images_dir, random_artist)))
    random_image_file = os.path.join(images_dir, random_artist, random_image)
    print (random_image_file)
    image = plt.imread(random_image_file)
    axes[i].imshow(image)
    axes[i].set_title("Artist: " + random_artist)
    axes[i].axis('off')

plt.show()

### Data Augmentation


Data Augmentation is a strategy that enables practitioners to significantly increase the diversity of data available for training models, without actually collecting new data. This technique like padding , cropping , shifting , flipping etc.

**ImageDataGenerator()** the ImageDataGenerator accepts the original data, randomly transforms it, and returns only the new, transformed data.

In [17]:
batch_size = 64
train_input_shape = (224, 224, 3)
n_classes = artists_top.shape[0]

train_datagen = ImageDataGenerator(validation_split=0.2,
                                   rescale=1./255.,
                                   #rotation_range=45,
                                   #width_shift_range=0.5,
                                   #height_shift_range=0.5,
                                   shear_range=5,
                                   #zoom_range=0.7,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                  )

train_generator = train_datagen.flow_from_directory(directory=images_dir,
                                                    class_mode='categorical',
                                                    target_size=train_input_shape[0:2],
                                                    batch_size=batch_size,
                                                    subset="training",
                                                    shuffle=True,
                                                    classes=artists_top_name.tolist()
                                                   )

valid_generator = train_datagen.flow_from_directory(directory=images_dir,
                                                    class_mode='categorical',
                                                    target_size=train_input_shape[0:2],
                                                    batch_size=batch_size,
                                                    subset="validation",
                                                    shuffle=True,
                                                    classes=artists_top_name.tolist()
                                                   )

STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size
print("Total number of batches =", STEP_SIZE_TRAIN, "and", STEP_SIZE_VALID)

### Build Model


So here in this part we are going to build model which train our data as previously I have mentioned that I will use state of the art technique like **ResNet50** model. I can use CNN(Convolutional Neural Network) but when I read the research paper that **ResNet50** network does a tremendous job on image data so let’s begin this section.

**ResNet50** model is also called identity layer why because the sole purpose of identity layer is skip-connection that means skip one layer in ResNet model which helps reducing vanishing gradient problem 

The ResNet model is adapted to the 1000 categories of ImageNet. Our task, however, is to classify some artists.

What can we do? With keras, it's easy to import only the convolutional part of VGG16, by setting the include_top parameter to False :

In [18]:
# Load pre-trained model
from keras.applications import *
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=train_input_shape)

for layer in base_model.layers:
    layer.trainable = True

In [19]:
base_model.summary()

ImageNet is an image database organized according to the WordNet hierarchy this is freely available for researcher and data scientist for research purposes.

### Create your Convolutional Neural Network

In [20]:
classifier = Flatten()(base_model.output)

#Initialize the CNN
classifier = Dense(512, activation='relu')(classifier)
classifier = BatchNormalization()(classifier)

classifier= Dense(16, activation='relu')(classifier)
classifier = BatchNormalization()(classifier)

output = Dense(n_classes, activation = 'softmax')(classifier)

In [21]:
model = Model(inputs=base_model.input, outputs=output)

### Compile the CNN

In [22]:
#Compile the CNN
model.compile(optimizer = Adam(lr =0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

### Train Model


In [23]:
n_epoch = 10

early_stop = EarlyStopping(monitor='val_loss', patience=20, verbose=1, 
                           mode='auto', restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, 
                              verbose=1, mode='auto')

In [24]:
history = model.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
                              validation_data=valid_generator, validation_steps=STEP_SIZE_VALID,
                              epochs=n_epoch,
                              shuffle=True,
                              verbose=1,
                              callbacks=[reduce_lr],
                              use_multiprocessing=True,
                              workers=16,
                              class_weight=class_weights
                             )

Here is the result after training the model for first time accuracy touched by my model is 0.98 or (98%) and loss reduce from 1.07 to 0.21 means model doing good now we freeze layers and re-train again.

### Fine tuning the model

Freezing a layer prevents its weights from being modified. This technique is often used in transfer learning, where the base model(trained on some other dataset)is frozen indicating that this layer should not be trained

In [ ]:
# Freeze core ResNet layers and train again 
for layer in model.layers:
    layer.trainable = False

for layer in model.layers[:50]:
    layer.trainable = True

optimizer = Adam(lr=0.0001)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, 
              metrics=['accuracy'])

n_epoch = 50
history2 = model.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
                              validation_data=valid_generator, validation_steps=STEP_SIZE_VALID,
                              epochs=n_epoch,
                              shuffle=True,
                              verbose=1,
                              callbacks=[reduce_lr, early_stop],
                              use_multiprocessing=True,
                              workers=16,
                              class_weight=class_weights
                             )

In [ ]:
plt.figure(figsize=(10, 10))

plt.subplot(2, 2, 1)
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Training - Loss Function')

plt.subplot(2, 2, 2)
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Train - Accuracy')

In [ ]:
# Prediction accuracy on train data
score = model.evaluate_generator(train_generator, verbose=1)
print("Prediction accuracy on train data =", score[1])

In [ ]:
score = model.evaluate_generator(valid_generator, verbose=1)
print("Prediction accuracy on test data =", score[1])

### print classification report

### Test Model

Here taking randomly 5 images and giving this random images to model to predict the artist by giving 5 random images that belong to particular artist or not.

In [ ]:
# Prediction
from keras.preprocessing import *

n = 4
fig, axes = plt.subplots(1, n, figsize=(25,10))

for i in range(n):
    random_artist = random.choice(artists_top_name)
    random_image = random.choice(os.listdir(os.path.join(images_dir, random_artist)))
    random_image_file = os.path.join(images_dir, random_artist, random_image)

    # Original image

    test_image = image.load_img(random_image_file, target_size=(train_input_shape[0:2]))

    # Predict artist
    test_image = image.img_to_array(test_image)
    test_image /= 255.
    test_image = np.expand_dims(test_image, axis=0)

    prediction = model.predict(test_image)
    prediction_probability = np.amax(prediction)
    prediction_idx = np.argmax(prediction)

    labels = train_generator.class_indices
    labels = dict((v,k) for k,v in labels.items())

    title = "Actual artist = {}\nPredicted artist = {}\nPrediction probability = {:.2f} %" \
                .format(random_artist.replace('_', ' '), labels[prediction_idx].replace('_', ' '),
                        prediction_probability*100)

    # Print image
    axes[i].imshow(plt.imread(random_image_file))
    axes[i].set_title(title)
    axes[i].axis('off')

plt.show()

So, from above we could see that given 4 random images our model predicted right artist name with given image on average probability of around 87% and above.

In [ ]:
model.save('model.keras')